<a href="https://colab.research.google.com/github/JJLimmm/optimizationresearch/blob/main/OR_Tools_LR_BnB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Operations Optimization II: Examples for Google OR-Tools

Here are some practice examples for the Operations Optimization II lessons on the _**Branch-and-Bound**_ and _**Heuristic**_ Algorithms.

You should finish the installation of **Google OR-Tools** and **Python** in your environment before starting on this practice code. The installation of Google OR-Tools can be found [here](https://developers.google.com/optimization/install)

Below are examples for Integer Programming (**IP**) cases in order to help you understand how to solve optimization programs with the OR-Tools library.

Here's the [guide](https://developers.google.com/optimization/mip) from Google OR-Tools documentation on Integer Programming Optimization.

==================================================================================================================

In [ ]:
#Install required libraries and dependencies
!python -m pip install -U ortools
!pip install pandas
!pip install prettytable

## Practice 1: Facility Location

In this example, we have binary variables included in our model. 

The problem description is as shown in the slides.

The formulation is
\begin{array}{rll}
                \min & \displaystyle \sum_{j=1}^{5} f_jx_j + \sum_{i=1}^{5} \sum_{j=1}^{5} c_{ij}y_{ij} & \\[15pt]
                \mbox{s.t.}  
                    & \displaystyle \sum_{i=1}^{5} y_{ij} \leq K_jx_j & \forall j = 1,...,5 \\[15pt]
                    & \displaystyle \sum_{j=1}^{5} y_{ij} \geq D_i & \forall i = 1,...,5 \\[15pt]
                    & x_{j} \in \{0, 1\} &\forall j = 1,...,5 \\[5pt]
                    & y_{ij} \geq 0 & \forall i = 1,...,5, j = 1,...,5.
\end{array}

where...
\begin{split}
    & f_j = \mbox{ weekly operation cost of distribution center j } \\
    & c_{ij} = \mbox{ Shipping cost per book from distribution center j to region i } \\
    & K_j = \mbox{ Capacity of distribution center j } \\
    & D_i = \mbox{ Book demand of region i }
\end{split}

and the decision variables are...
\begin{split}
    & x_j = \mbox{ 1 if a distribution center is built at location j, 0 otherwise. } \\
    & y_{ij} = \mbox{ Number of books shipped from distribution center j (cities) to region i (markets) }
\end{split}

### Step 1
We should import the Google OR-Tools optimization package and its relevant libraries called *ortools* first, and then declare the **solver**.

However, this time round instead of using the GLOP, we shall use the **SCIP** (Solving Constraints Integer Programs) for Mixed Integer (Linear and Nonlinear) Programming (**MIP**) when declaring the solver.

The *pywraplp* is a Python wrapper for the underlying solver created in C++.
The [API Reference Documentation](https://google.github.io/or-tools/python/ortools/linear_solver/pywraplp.html#Solver) can be found here.

In [ ]:
# Import libraries and packages
from ortools.linear_solver import pywraplp
from ortools.init import pywrapinit
import pandas as pd

# Declare the solver
# The argument 'SCIP' is for using the Solving Constraints Integer Programs back-end package
solver = pywraplp.Solver.CreateSolver('SCIP')

#### Next, we will load the data from the excel sheets. 

In [ ]:
# Read excel sheets and transform them into lists and matrices

#Read from 1st sheet, Basic Information
#where 
#cities = distribution centers j
#market = regions i
basic_info = pd.read_excel('IP_dataset.xlsx', 'Basic information', engine='openpyxl')
cities = range(len(basic_info['City']))
city_names = basic_info['City']
markets = range(len(basic_info['Market']))
market_names = basic_info['Market']

#Read from 2nd sheet, City's Information
city_info = pd.read_excel('IP_dataset.xlsx', 'City\'s information', engine='openpyxl')
operating_costs = city_info['Operating cost']
capacities = city_info['Capacity']

#Read from 3rd sheet, Market's Information
market_info = pd.read_excel('IP_dataset.xlsx', 'Market\'s information', engine='openpyxl')
demands = market_info['Demand']

#Read from 4th sheet, Shipping cost
shipping_info = pd.read_excel('IP_dataset.xlsx', 'Shipping cost', index_col = 0, engine='openpyxl')
shipping_costs = []
for i in shipping_info.index:
    shipping_costs.append(list(shipping_info.loc[i]))

#### Run the portion of the code below if you want to see how the data looks like when stored.

In [ ]:
# Display data loaded from excel with print statements
print('Data from "Basic Information" sheet.\n')
with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
    print(basic_info)
print("\n cities: ", cities)
print("\n City names: \n", city_names)
print("\n markets: ", markets)
print("\n Market names: \n", market_names)

print('\n\n Data from "City\'s Information" sheet.\n\n')
with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
    print(city_info)
print("\n operating_costs: \n", operating_costs)
print(type(operating_costs))
print("\n capacities: \n", capacities)

print('\n\n Data from "Market\'s Information" sheet.\n')
with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
    print(market_info)
print("\n demands: \n", demands)

print('\n\n Data from "Shipping cost" sheet.')
with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
    print(shipping_info)
print("\n shipping_costs: \n", shipping_costs)

Data from "Basic Information" sheet.

               City     Market
0       Spokane, WA  Northwest
1          Reno, NV  Southwest
2         Omahe, NE    Midwest
3    Harrisburg, PA  Southeast
4  Jacksonville, FL  Northeast

 cities:  range(0, 5)

 City names: 
 0         Spokane, WA
1            Reno, NV
2           Omahe, NE
3      Harrisburg, PA
4    Jacksonville, FL
Name: City, dtype: object

 markets:  range(0, 5)

 Market names: 
 0    Northwest
1    Southwest
2      Midwest
3    Southeast
4    Northeast
Name: Market, dtype: object


 Data from "City's Information" sheet.


               City  Operating cost  Capacity
0       Spokane, WA           40000     20000
1          Reno, NV           30000     20000
2         Omahe, NE           25000     15000
3    Harrisburg, PA           40000     25000
4  Jacksonville, FL           30000     15000

 operating_costs: 
 0    40000
1    30000
2    25000
3    40000
4    30000
Name: Operating cost, dtype: int64
<class 'pandas.core.series

### Step 2

Now, you will attempt to create your Integer Programming Solver for this practice example with the Model-Data Decoupling concept.

Using the same steps as what you have learnt during the Practice 1 for Simplex Method, create your program and solve the problem.

#### Recall:

The steps for creating an IP solver is similar to the steps for a LP solver in simplex method where...
1. Import the ortools library and declare the **Solver**.
2. Declare **Variables** in LP problem.
3. Declare the **Objective function**.
4. Declare the **Constraints**.
5. Call the ortools solver to find the **Optimal BFS**.
6. Check the **Solver status** and display the **results**.

**Step 1** has already been done above. Continue on with **step 2** to create your solver.

In [ ]:
### Start Here ###

## Step 2: Declare Variables in the IP problem
# hint1: variables are the decision variables from the example
# hint2: For creating binary integer variables, declare the lb and ub when using IntVar() as 0 and 1, respectively.
# Initialise two lists to store the 2 variables
x = []
y = []
# Declare and store variables of x and y in its list
for j in cities:
    x.append(solver.IntVar(lb = 0, ub = 1, name = "x" + str(j+1)))
for i in markets:
    y.append([])  # creating a nested list; for each i, there is a list of j
    for j in cities:
        y[i].append(solver.NumVar(lb = 0, ub = solver.infinity(), name = "y" + str(i+1) + str(j+1)))
print("Number of variables : ", solver.NumVariables())
print("\n x variables: ",x)
print("\n y variables: ",y)


## Step 3: Declare the objective function
obj_func = solver.Objective()
for j in cities:
    obj_func.SetCoefficient(x[j] , operating_costs[j].item())
for i in markets:
    for j in cities:
        obj_func.SetCoefficient(y[i][j], shipping_costs[i][j])


## Step 4: Declare the Constraints
#hint1: Think of how many constraints there are
for j in cities:
    solver.Add(sum(y[i][j] for i in markets) <= (capacities[j] * x[j])) # Capacity constraint

for i in markets:
    solver.Add(sum(y[i][j] for j in cities) >= demands[i]) # Demands Constraint

print("\n Number of Constraints: ", solver.NumConstraints())


## Step 5: Call the ortools solver and find the Optimal solution
solver_status = solver.Solve()
print("\nSolver has finished.")


## Step 6: Check Solver status and display results
if solver_status == pywraplp.Solver.OPTIMAL:
    print(f"\nThe flag number for the solver status is {solver_status}")
    print("---------------------------------------------------------\n")
    print("Result:\n")
    for j in cities:
        print(x[j].name(), '=', x[j].solution_value())
        
    # Print out a table for results using the PrettyTable library functions
    from prettytable import PrettyTable
    results_table = PrettyTable()
    column0 = ["Centers"]
    headers = pd.concat([pd.Series(column0), market_names], ignore_index =True)
    results_table.field_names = headers
    for j in cities:
        row_values = []
        row_values.append(city_names[j])
        for i in markets:
            row_values.append(round(y[i][j].solution_value(),1))
        results_table.add_row(row_values)
    print(results_table)
    
    # Display other metrics
    print("Problem solved in {:.2f} ms.".format(solver.wall_time()))
    print("Objective Value, z* =", solver.Objective().Value())
    print("No. of Iterations=", solver.Iterations())
    print("No. of nodes explored=", solver.nodes())

else:
    print(f"The problem does not have an optimal solution with Flag number {solver_status}.")


Number of variables :  30

 x variables:  [x1, x2, x3, x4, x5]

 y variables:  [[y11, y12, y13, y14, y15], [y21, y22, y23, y24, y25], [y31, y32, y33, y34, y35], [y41, y42, y43, y44, y45], [y51, y52, y53, y54, y55]]

 Number of Constraints:  10

Solver has finished.

 The flag number for the solver status is 0
---------------------------------------------------------

Result:

x1 = 0.0
x2 = 1.0
x3 = 0.0
x4 = 1.0
x5 = 1.0
+------------------+-----------+-----------+---------+-----------+-----------+
|     Centers      | Northwest | Southwest | Midwest | Southeast | Northeast |
+------------------+-----------+-----------+---------+-----------+-----------+
|   Spokane, WA    |    0.0    |    0.0    |   0.0   |    0.0    |    0.0    |
|     Reno, NV     |   8000.0  |  12000.0  |   0.0   |    0.0    |    0.0    |
|    Omahe, NE     |    0.0    |    0.0    |   0.0   |    0.0    |    0.0    |
|  Harrisburg, PA  |    0.0    |    0.0    |  8000.0 |    0.0    |  17000.0  |
| Jacksonville, FL |   

### What if...
Now the company wants to set up at least 4 centers for some reasons, how should we modify the model?